In [4]:
%matplotlib inline
pd.options.display.max_columns = 50

import sys
sys.path.append('/Users/sinsanghun/anaconda/lib/python2.7/site-packages')

%cd nba

[Errno 2] No such file or directory: 'nba'
/Users/sinsanghun/nba


In [5]:
year1 = ["97", "98", "99", "00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16" ]
year2 = ["98", "99", "00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17" ]

season = []
for i in range(20):
    season.append(year1[i] + year2[i])
    

In [6]:
df = []
for i in season:
    df.append(pd.read_csv(i+".csv"))

In [7]:
for i in range(20):
    df[i] = df[i][["Pos", "GS", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "2P", "2PA", "2P%", "eFG%", "FT", "FTA", "FT%",
          "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PS/G"]]

In [8]:
temp = df[0]

# 9798 ~ 1516 년도 데이터를 train set으로 쓰고 
# 1617 데이터를 test set으로 사용하자 

for i in range(18):
    temp = pd.concat([temp, df[i+1]])
    
temp.shape

(10739, 24)

In [9]:
# index 정렬 
position = temp.set_index(np.arange(len(temp)))

In [10]:
position2 = position[(position["Pos"] == "C") | (position["Pos"] == "PF") |(position["Pos"] == "SF") | (position["Pos"] == "SG") |(position["Pos"] == "PG")]

In [29]:
position3 = position2.dropna()

In [12]:
dfy = position3[["Pos"]]
dfX = position3.iloc[:,1:]

In [13]:
test = df[19].dropna()

In [14]:
test = test[(test["Pos"] == "C") | (test["Pos"] == "PF") \
                     |(test["Pos"] == "SF") | (test["Pos"] == "SG") |(test["Pos"] == "PG")]

In [15]:
test_y = test[["Pos"]]
test_X = test.iloc[:,1:]

In [16]:
dfy[dfy == "SF"] = "F"
dfy[dfy == "PF"] = "F"
dfy[dfy == "SG"] = "G"
dfy[dfy == "PG"] = "G"

test_y[test_y == "SF"] = "F"
test_y[test_y == "PF"] = "F"
test_y[test_y == "SG"] = "G"
test_y[test_y == "PG"] = "G"

/Users/sinsanghun/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sinsanghun/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/pandas/core/frame.py:2366: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)
/Users/sinsanghun/anaconda/envs/ipykernel_py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

-------------------------

## 나이브 베이지안

In [37]:
from sklearn.naive_bayes import MultinomialNB
clf= MultinomialNB().fit(dfX, dfy)

In [38]:
from sklearn.metrics import confusion_matrix

y_true = test_y
y_pred = clf.predict(test_X)
confusion_matrix(y_true, y_pred)

array([[ 27,  45,   2],
       [ 18, 128,  74],
       [  1,  16, 219]])

In [39]:
from sklearn.metrics import *

target_names = ["C", "F", "G"]
print(classification_report(y_true, y_pred, target_names=target_names))
print(accuracy_score(y_true, y_pred))

             precision    recall  f1-score   support

          C       0.59      0.36      0.45        74
          F       0.68      0.58      0.63       220
          G       0.74      0.93      0.82       236

avg / total       0.69      0.71      0.69       530

0.705660377358


## LDA

In [30]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True).fit(dfX, dfy)

In [33]:
from sklearn.metrics import confusion_matrix

y_true = test_y
y_pred = lda.predict(test_X)
confusion_matrix(y_true, y_pred)

array([[ 40,  34,   0],
       [ 17, 155,  48],
       [  0,  27, 209]])

In [34]:
from sklearn.metrics import *

target_names = ["C", "F", "G"]
print(classification_report(y_true, y_pred, target_names=target_names))
print(accuracy_score(y_true, y_pred))

             precision    recall  f1-score   support

          C       0.70      0.54      0.61        74
          F       0.72      0.70      0.71       220
          G       0.81      0.89      0.85       236

avg / total       0.76      0.76      0.76       530

0.762264150943


## 서포트벡터머신

In [40]:
from sklearn.svm import SVC
model = SVC(kernel='linear').fit(dfX, dfy)

In [41]:
from sklearn.metrics import confusion_matrix
y_true = test_y
y_pred = model.predict(test_X)
confusion_matrix(y_true, y_pred)

array([[ 31,  43,   0],
       [  8, 178,  34],
       [  0,  37, 199]])

In [42]:
from sklearn.metrics import *

target_names = ["C", "F", "G"]
print(classification_report(y_true, y_pred, target_names=target_names))
print(accuracy_score(y_true, y_pred))

             precision    recall  f1-score   support

          C       0.79      0.42      0.55        74
          F       0.69      0.81      0.74       220
          G       0.85      0.84      0.85       236

avg / total       0.78      0.77      0.76       530

0.769811320755


## SGD classifier

In [43]:
from sklearn.linear_model import SGDClassifier
modelSGD = SGDClassifier(loss="log", n_iter=1000, random_state=1).fit(dfX, dfy)

In [44]:
from sklearn.metrics import confusion_matrix
y_true = test_y
y_pred = modelSGD.predict(test_X)
confusion_matrix(y_true, y_pred)

array([[ 30,  41,   3],
       [  8, 152,  60],
       [  0,  23, 213]])

In [45]:
from sklearn.metrics import *

target_names = ["C", "F", "G"]
print(classification_report(y_true, y_pred, target_names=target_names))
print(accuracy_score(y_true, y_pred))

             precision    recall  f1-score   support

          C       0.79      0.41      0.54        74
          F       0.70      0.69      0.70       220
          G       0.77      0.90      0.83       236

avg / total       0.75      0.75      0.73       530

0.745283018868


## XGboost

In [149]:
import xgboost
model_xgb = xgboost.XGBClassifier(n_estimators=100, max_depth=8, gamma=0.15).fit(dfX, dfy)

In [156]:
from sklearn.metrics import confusion_matrix
y_true = 
y_pred = model_xgb.predict(dfX)
confusion_matrix(y_true, y_pred)

array([[1038,   13,    0],
       [  12, 3297,   46],
       [   0,   26, 3946]])

In [157]:
from sklearn.metrics import *

target_names = ["C", "F", "G"]
print(classification_report(y_true, y_pred, target_names=target_names))
print("accuracy : ", accuracy_score(y_true, y_pred))

             precision    recall  f1-score   support

          C       0.99      0.99      0.99      1051
          F       0.99      0.98      0.99      3355
          G       0.99      0.99      0.99      3972

avg / total       0.99      0.99      0.99      8378

accuracy :  0.98842205777


------------

# 모형 최적화 

In [146]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

pipe_svc = Pipeline([('clf', xgboost.XGBClassifier(n_estimators=100))])

param_range = [0.05, 0.1, 0.15]
param_depth = [7, 8, 9]
param_grid = [
    {'clf__gamma' : param_range, 'clf__max_depth' : param_depth }]

gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=10, n_jobs=4)

gs.fit(np.array(dfX), np.array(dfy).flatten())

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('clf', XGBClassifier(base_score=0.5, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, nthread=-1, objective='binary:logistic', seed=0,
       silent=True, subsample=1))]),
       fit_params={}, iid=True, n_jobs=4,
       param_grid=[{'clf__gamma': [0.05, 0.1, 0.15], 'clf__max_depth': [7, 8, 9]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [147]:
gs.cv_results_["mean_test_score"]

array([ 0.80138458,  0.7998329 ,  0.79768441,  0.80066842,  0.79553593,
        0.79840057,  0.79911673,  0.80257818,  0.79995226])

In [148]:
print(gs.best_score_)
print(gs.best_params_)

0.80257818095
{'clf__gamma': 0.15, 'clf__max_depth': 8}


----

---- 

In [22]:
import tensorflow as tf

In [28]:
from keras.utils import np_utils

ImportError: cannot import name utils